In [1]:
from converters import AlignWrapper
import torchvision
import torch
from torch import nn
import copy
# m is a standard pytorch model
m1 = torchvision.models.resnet18()
# m1 = m1.features
m1 = nn.DataParallel(m1)
# after convert, m is using AlignConv
conv_cfg = dict(conv_type='AlignConv',padding_mode='zeros', 
                 n_fold=8, align=True, inplace=True,
                 ref_spacing=0.2, shift_padding_zero=True)
wrapper = AlignWrapper(conv_cfg)
m2 = wrapper(copy.deepcopy(m1))

In [2]:
batch_size = 4
in_channels = 1
d = 7
height = 320
width = 320
input = torch.rand(batch_size, in_channels, d,height, width).cuda()
spacing = torch.rand(batch_size, 1).cuda()
m2.cuda()
out = m2(input, z_spacing=spacing)

tensor([[0.2399],
        [0.0817]], device='cuda:1')
tensor([[0.1586],
        [0.9330]], device='cuda:0')
tensor([[0.2399],
        [0.0817]], device='cuda:1')
tensor([[0.1586],
        [0.9330]], device='cuda:0')
tensor([[0.2399],
        [0.0817]], device='cuda:1')
tensor([[0.1586],
        [0.9330]], device='cuda:0')
tensor([[0.2399],
        [0.0817]], device='cuda:1')
tensor([[0.1586],
        [0.9330]], device='cuda:0')
tensor([[0.2399],
        [0.0817]], device='cuda:1')
tensor([[0.1586],
        [0.9330]], device='cuda:0')
tensor([[0.2399],
        [0.0817]], device='cuda:1')
tensor([[0.1586],
        [0.9330]], device='cuda:0')
tensor([[0.2399],
        [0.0817]], device='cuda:1')
tensor([[0.1586],
        [0.9330]], device='cuda:0')
tensor([[0.2399],
        [0.0817]], device='cuda:1')
tensor([[0.1586],
        [0.9330]], device='cuda:0')
tensor([[0.2399],
        [0.0817]], device='cuda:1')
tensor([[0.1586],
        [0.9330]], device='cuda:0')
tensor([[0.2399],
        [0

In [4]:
out.shape

torch.Size([4, 1000])